<a href="https://colab.research.google.com/github/Firojpaudel/Reasoning_Model_/blob/main/Creating_reasoning_model_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Before Diving In:

**What is GRPO?**
- Stands for Group Relative Policy Optimization. Its is a reinforcement learning technique that helps models develop reasoning capabilities autonomously.
- First introduced the concept on DeepSeek paper.

---

#### **Why Unsloth?**:
- Well, thanks to Unsloth, training a larger params model ( $15B$ or so) on customer-grade GPUs like that of Colab and Kaggle is possible due to some major optimizations done on the **transformers** library.